## Embedding - Chroma

Dieses Notebook wird verwendet, um die ChromaEmbedding Pipeline zu erstellen und zu testen.

In [8]:
import pandas as pd
import numpy as np
import os
from langchain_openai import AzureOpenAIEmbeddings
from langchain.vectorstores import Chroma
from utilities import TextCleaner
from credentials import AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT
from tqdm import tqdm

### Daten einlesen

In [2]:
path = '/Users/husazwerg/code/gitlab.fhnw.ch/gruppen_arbeit/npr_fs25/MC1/data_mc1/cleantech_media_dataset_v3_2024-10-28.csv'
df_train = pd.read_csv(path)

df_train.head()

,Unnamed: 0,title,date,author,content,domain,url
0,93320,"XPeng Delivered ~100,000 Vehicles In 2021",2022-01-02,NaN,['Chinese automotive startup XPeng has shown o...,cleantechnica,https://cleantechnica.com/2022/01/02/xpeng-del...
1,93321,Green Hydrogen: Drop In Bucket Or Big Splash?,2022-01-02,NaN,['Sinopec has laid plans to build the largest ...,cleantechnica,https://cleantechnica.com/2022/01/02/its-a-gre...
2,98159,World’ s largest floating PV plant goes online...,2022-01-03,NaN,['Huaneng Power International has switched on ...,pv-magazine,https://www.pv-magazine.com/2022/01/03/worlds-...
3,98158,Iran wants to deploy 10 GW of renewables over ...,2022-01-03,NaN,"['According to the Iranian authorities, there ...",pv-magazine,https://www.pv-magazine.com/2022/01/03/iran-wa...
4,31128,Eastern Interconnection Power Grid Said ‘ Bein...,2022-01-03,NaN,['Sign in to get the best natural gas news and...,naturalgasintel,https://www.naturalgasintel.com/eastern-interc...


### Text Preprocessing

In [3]:
# Text preprocessing
cleaner = TextCleaner()
df_cleaned = cleaner.clean_text_column(df_train, column='content')


Wichtig ist auch zu wissen, wie viele Tokens wir haben, da vortrainierte Modelle eine maximale Anzahl an Tokens haben. Azure OpenAI hat eine maximale Anzahl von etwa 8200 Tokens.

In [ ]:
# Wörter zählen
df_cleaned["word_count"] = df_cleaned["content"].astype(str).apply(lambda x: len(x.split()))

# Tokenanzahl schätzen (auf Basis von Wörtern)
df_cleaned["estimated_tokens"] = df_cleaned["word_count"] * 1.3

# Vorschau
print(df_cleaned[["content", "word_count", "estimated_tokens"]].head())
total_tokens = df_cleaned["estimated_tokens"].sum()
print(f"Gesamtanzahl geschätzter Tokens: {int(total_tokens):,}")


                                              content  word_count  \
6   'BP' s " long term " commitment to Scotland is...        1117   
9   'Israeli researchers have tested organic PV mo...         601   
14  'High wind loads increase structural design co...         497   
19  " More info.", "Anthropogenic emissions of car...         495   
21  'While the battle between utilities and reside...         285   

    estimated_tokens  
6             1452.1  
9              781.3  
14             646.1  
19             643.5  
21             370.5  
📊 Gesamtanzahl geschätzter Tokens: 3,143,657


/var/folders/0l/jk3grg717z34yf9xh85kj0bh0000gn/T/ipykernel_13458/1774651688.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["word_count"] = df_cleaned["content"].astype(str).apply(lambda x: len(x.split()))
/var/folders/0l/jk3grg717z34yf9xh85kj0bh0000gn/T/ipykernel_13458/1774651688.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["estimated_tokens"] = df_cleaned["word_count"] * 1.3


### Chunking

Um keine wichtigen Informationen zu verlieren, ist es wichtig den Text in Chunks zu erlegen (wollen die wichtigen Token behalten)

### Embedding

In [4]:
os.environ["AZURE_OPENAI_API_KEY"] = credentials.AZURE_OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = credentials.AZURE_OPENAI_ENDPOINT
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-05-15"

Ich werde hier das Model text-embedding-3-large verwenden und schauen, ob die Embeddings erstellt werden

In [8]:
# Embedding Model Azure OpenAI 
# Model: text-embedding-3-large
embedding_model_1 = AzureOpenAIEmbeddings(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    deployment="text-embedding-3-large", 
)

In [ ]:
# embeddings erstellen (mit Pausen, aufgrund von Rate-Limits)
import time

texts_model_1 = df_cleaned["content"].tolist()

# Embeddings berechnen
vectors = []
for i, text in enumerate(texts_model_1):
    vector_model_1 = embedding_model_1.embed_query(text)
    vectors.append(vector_model_1)
    
    if i % 3 == 0: # alle 3 Abfragen 1 Sekunde Pause
        time.sleep(1.5)

# Kontrollausgabe
print("Erster Text:", texts_model_1[0][:100], "...")
print("Erster Embedding-Vektor (gekürzt):", vector_model_1[0][:5])

In [ ]:
# Test mit kleiner Teilmenge, dauert sonst zu lange
texts_sample = df_cleaned["content"].dropna().astype(str).tolist()[:10]

# Embeddings berechnen
vectors_sample_1 = embedding_model_1.embed_documents(texts_sample)

# Ausgabe
print("Erster Text (gekürzt):", texts_sample[0][:100])
print("Erster Embedding-Vektor (gekürzt):", vectors_sample_1[0][:5])


Erster Text (gekürzt): 'Chinese automotive startup XPeng has shown one of the most dramatic auto production ramp ups in his
Erster Embedding-Vektor (gekürzt): [0.004027563147246838, 0.027820918709039688, -0.020796367898583412, -0.010111656039953232, -0.010490612126886845]


### ChromaDB

Es wird nun mal ein Prototyp einer ChromaDB erstellt, mit welcher wir weiter das RAG-Modell erstellen können. In unserem letzten Schritt werden wir dann verschiedene Parameter testen und somit auch mehrere DB erstellen.

In [5]:
os.environ["AZURE_OPENAI_API_KEY"] = credentials.AZURE_OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = credentials.AZURE_OPENAI_ENDPOINT
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-05-15"

In [ ]:
from embedding import generate_embeddings_to_chroma

vector_store = generate_embeddings_to_chroma(
    df=df_cleaned,
    text_column="content",
    deployment_name="text-embedding-3-large",
    batch_size=10,
    sleep_time=2,
    chunk_size=800,
    chunk_overlap=100
)


Insgesamt 24453 Chunks erzeugt.
✅ Batch 1: 10 Chunks hinzugefügt
✅ Batch 2: 10 Chunks hinzugefügt
✅ Batch 3: 10 Chunks hinzugefügt
✅ Batch 4: 10 Chunks hinzugefügt
✅ Batch 5: 10 Chunks hinzugefügt
✅ Batch 6: 10 Chunks hinzugefügt
✅ Batch 7: 10 Chunks hinzugefügt
✅ Batch 8: 10 Chunks hinzugefügt
✅ Batch 9: 10 Chunks hinzugefügt
✅ Batch 10: 10 Chunks hinzugefügt
✅ Batch 11: 10 Chunks hinzugefügt
✅ Batch 12: 10 Chunks hinzugefügt
✅ Batch 13: 10 Chunks hinzugefügt
✅ Batch 14: 10 Chunks hinzugefügt
✅ Batch 15: 10 Chunks hinzugefügt
✅ Batch 16: 10 Chunks hinzugefügt
✅ Batch 17: 10 Chunks hinzugefügt
✅ Batch 18: 10 Chunks hinzugefügt
✅ Batch 19: 10 Chunks hinzugefügt
✅ Batch 20: 10 Chunks hinzugefügt
✅ Batch 21: 10 Chunks hinzugefügt
✅ Batch 22: 10 Chunks hinzugefügt
✅ Batch 23: 10 Chunks hinzugefügt
✅ Batch 24: 10 Chunks hinzugefügt
✅ Batch 25: 10 Chunks hinzugefügt
✅ Batch 26: 10 Chunks hinzugefügt
✅ Batch 27: 10 Chunks hinzugefügt
✅ Batch 28: 10 Chunks hinzugefügt
✅ Batch 29: 10 Chunks hin

#### ChromaDB laden

In [5]:
# Embedding-Funktion (muss identisch sein wie beim Erstellen der DB!)
embedding_model = AzureOpenAIEmbeddings(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    deployment="text-embedding-3-large" # muss mit dem Deployment-Namen übereinstimmen
)

# Ordnerpfad, wo deine .bin/.pickle-Dateien liegen
persist_directory = "MC1/src/chroma_langchain_db/1180bfbe-420d-4671-8e5a-d2c56960e7b7"

# Collection-Namen angeben
collection_name = "text-embedding-3-large" # wird automatisch erstellt, haben wir also

# Chroma laden
vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=collection_name,
    persist_directory=persist_directory
)

/var/folders/0l/jk3grg717z34yf9xh85kj0bh0000gn/T/ipykernel_44954/2905731710.py:16: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [9]:
count = vector_store._collection.count()
print(f"Anzahl der gespeicherten Dokumente: {count}")


Anzahl der gespeicherten Dokumente: 0


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from Chroma_Embedding_Pipeline import ChromaEmbeddingPipeline

In [ ]:
df_train_cleaned = pd.read_parquet('../data_mc1/data_processed/df_train_1000.parquet')

In [ ]:
# Vorlage für das erstellen einer ChromaDB
# 1) TextSplitter optional definieren
my_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " ", ""]
)

# 2) Metadaten-Dict für Chroma (optional)
metadata_config = {
    "hnsw:space": "cosine",
    "hnsw:M": 16,
    "efConstruction": 200
}

# 3) Pipeline-Objekt erstellen
pipeline = ChromaEmbeddingPipeline(
    df=df_200_cleaned,
    text_column="content",
    deployment_name="text-embedding-3-large",  # Dein Azure-Deployment-Name
    splitter=my_splitter,
    collection_name="auto",                    # Automatisch generiert
    persist_directory="auto",                  # Automatisch generiert => z.B. "./chroma_recchar_cs800_co100_3-large"
    metadata=metadata_config,
    batch_size=20,
    sleep_time=1.0
)

# 4) Pipeline starten -> Chroma-DB wird erstellt
vector_store = pipeline.run()


In [ ]:
persist_directory = "./chroma_recchar_cs800_co100_3-large"  
collection_name = "recchar_cs800_co100_3-large"  # oder so ähnlich

# 4. Dokumente zählen (interner Zugriff, für Debug-Zwecke)
count = vector_store._collection.count()
print(f"Dokumente in der Collection '{collection_name}': {count}")

# Oder eine kleine Stichprobe anzeigen
docs_info = vector_store._collection.peek(limit=3)  # holt 3 Dokumente
print(docs_info)

In [ ]:
# Embedding-Funktion (muss identisch sein wie beim Erstellen der DB!)
embedding_model = AzureOpenAIEmbeddings(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2023-05-15",
    deployment="text-embedding-3-large"  # Muss mit dem Deployment-Namen übereinstimmen
)

# Dynamischer Basis-Pfad finden
# Angenommen, das Notebook liegt in npr_fs25/MC1/doc
notebook_dir = Path.cwd()  # Aktuelles Arbeitsverzeichnis (doc)
project_root = notebook_dir.parent  # Eine Ebene höher (npr_fs25/MC1)
src_dir = project_root / "src"  # Pfad zu npr_fs25/MC1/src

# Stelle sicher, dass src_dir existiert
if not src_dir.exists():
    raise FileNotFoundError(f"Source directory {src_dir} does not exist. Please check the project structure.")

# Konfigurationen der zu ladenden Datenbanken
db_configs = [
    {
        "persist_directory": str(src_dir / "chroma_recchar_800_cosine"),
        "collection_name": "recchar_800_cosine",
        "alt_collection_name": "chroma_recchar_800_cosine"
    },
    {
        "persist_directory": str(src_dir / "chroma_recchar_800_l2"),
        "collection_name": "recchar_800_l2",
        "alt_collection_name": "chroma_recchar_800_l2"
    },
    {
        "persist_directory": str(src_dir / "chroma_recchar_800_ip"),
        "collection_name": "recchar_800_ip",
        "alt_collection_name": "chroma_recchar_800_ip"
    },
    {
        "persist_directory": str(src_dir / "chroma_senttrans_200_cosine"),
        "collection_name": "senttrans_200_cosine",
        "alt_collection_name": "chroma_senttrans_200_cosine"
    },
    {
        "persist_directory": str(src_dir / "chroma_senttrans_200_l2"),
        "collection_name": "senttrans_200_l2",
        "alt_collection_name": "chroma_senttrans_200_l2"
    },
    {
        "persist_directory": str(src_dir / "chroma_senttrans_200_ip"),
        "collection_name": "senttrans_200_ip",
        "alt_collection_name": "chroma_senttrans_200_ip"
    }
]

# Dictionary zum Speichern der geladenen Vector Stores
vector_stores = {}

for config in db_configs:
    try:
        client = chromadb.PersistentClient(path=str(config["persist_directory"]))
        collection_names = client.list_collections()
        
        if config["collection_name"] in collection_names:
            print(f"Loading {config['collection_name']}...")
            vector_store = Chroma(
                embedding_function=embedding_model,
                collection_name=config["collection_name"],
                persist_directory=str(config["persist_directory"])
            )
            count = vector_store._collection.count()
            print(f"  -> {count} Dokumente geladen.\n")
            vector_stores[config["collection_name"]] = vector_store
        else:
            print(f"Collection {config['collection_name']} nicht gefunden, übersprungen.\n")
    except Exception as e:
        print(f"Fehler beim Laden von {config['collection_name']}: {str(e)}\n")

# Zusammenfassung
print("\nGeladene Vector Stores:", list(vector_stores.keys()))

In [ ]:
# Embedding-Funktion (muss identisch sein wie beim Erstellen der DB!)
embedding_model = AzureOpenAIEmbeddings(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2023-05-15",
    deployment="text-embedding-3-large"  # Muss mit dem Deployment-Namen übereinstimmen
)

# Pfad zur zu ladenden DB
persist_directory = Path.cwd().parent / "src" / "chroma_recchar_800_cosine"

collection_name = "recchar_800_cosine"

# Client und Collection prüfen
client = chromadb.PersistentClient(path=str(persist_directory))
collection_names = client.list_collections()

if collection_name not in collection_names:
    raise ValueError(f"Collection '{collection_name}' nicht gefunden in {persist_directory}.")

print(f"Collection '{collection_name}' gefunden. Laden...")

# Vector Store laden
vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=collection_name,
    persist_directory=str(persist_directory)
)

# Dokument-Anzahl prüfen
doc_count = vector_store._collection.count()
print(f"Datenbank geladen: {doc_count} Dokumente gefunden.")

### Evaluierung

In [2]:
df_evaluierung = pd.read_parquet('../data_mc1/data_eval/df_results_similarity_simple.parquet')

In [3]:
df_evaluierung.head()

,question,relevant_text,relevant_text_llm,retrieved_context,answer,answer_llm
0,What is the EU’s Green Deal Industrial Plan?,The European counterpart to the US Inflation R...,[European Commission introduces Green Deal Ind...,European Commission introduces Green Deal Indu...,The EU’s Green Deal Industrial Plan aims to en...,The EU’s Green Deal Industrial Plan is a strat...
1,When did the cooperation between GM and Honda ...,What caught our eye was a new hookup between G...,[. The Army demonstration was built on GM' s P...,. The Army demonstration was built on GM' s Pr...,July 2013,The cooperation between GM and Honda on fuel c...
2,Did Colgate-Palmolive enter into PPA agreement...,"Scout Clean Energy, a Colorado-based renewable...","[Scout, Colgate-Palmolive Sign PPA for Texas S...","Scout, Colgate-Palmolive Sign PPA for Texas So...",yes,"Yes, Colgate-Palmolive entered into a power pu..."
3,What is the status of ZeroAvia's hydrogen fuel...,"In December, the US startup ZeroAvia announced...","[. "" The 19 seat twin engine aircraft has been...",". "" The 19 seat twin engine aircraft has been ...",ZeroAvia's hydrogen fuel cell electric aircraf...,"Based on the context provided, ZeroAvia's hydr..."
4,"What is the ""Danger Season""?",As spring turns to summer and the days warm up...,[. Over the past 30 40 years wildfire data sho...,. Over the past 30 40 years wildfire data show...,"The ""Danger Season"" is the period in the North...","The ""Danger Season"" refers to the period in th..."


In [7]:
# Initialisiere Embedding-Modell
embedding_model = AzureOpenAIEmbeddings(
	azure_endpoint=AZURE_OPENAI_ENDPOINT,
	api_key=AZURE_OPENAI_API_KEY,
	api_version="2023-05-15",
	deployment="text-embedding-ada-002"
)

In [10]:
# 3. Liste der Felder, die eingebettet werden sollen
fields_to_embed = ['question', 'answer', 'answer_llm', 'retrieved_context']

# 4. Funktion zum Einbetten mit Fortschrittsanzeige
def embed_texts(texts):
    embeddings = []
    batch_size = 50  # anpassen je nach API-Limit
    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding batches"):
        batch = texts[i:i+batch_size]
        batch_embeddings = embedding_model.embed_documents(batch)
        embeddings.extend(batch_embeddings)
    return embeddings

# 5. Einbetten für jedes Feld
df_embedded = df_evaluierung.copy()  # wichtig: Original nicht überschreiben

for field in fields_to_embed:
    print(f"Embedding {field}...")
    texts = df_embedded[field].astype(str).tolist()  # sicherstellen, dass es Strings sind
    df_embedded[field + '_embedding'] = embed_texts(texts)

# 6. als Parameter speichern
output = df_embedded


Embedding question...


Embedding batches: 100%|██████████| 1/1 [00:00<00:00,  2.07it/s]


Embedding answer...


Embedding batches: 100%|██████████| 1/1 [00:00<00:00,  6.35it/s]


Embedding answer_llm...


Embedding batches: 100%|██████████| 1/1 [00:00<00:00,  4.10it/s]


Embedding retrieved_context...


Embedding batches: 100%|██████████| 1/1 [00:00<00:00,  2.69it/s]
